<a href="https://colab.research.google.com/github/WoleIlori/Data-Preparation-and-Visualisation/blob/master/Data_Preparation_for_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Using a python wrapper to obtain dataset
from sodapy import Socrata
import calendar
import json
import pandas as pd
from collections import defaultdict
import string

In [0]:
data_source = "data.sfgov.org"
client = Socrata(data_source, None)

In [0]:
#MORE_INFO: https://dev.socrata.com/foundry/data.sfgov.org/5m6g-bqm4
def get_write_data(file_path, api_key,  month_name):
    raw_data = client.get(api_key, month= month_name)
    
    raw_json=json.dumps(raw_data)
    
    #strip array brackets
    raw_json = raw_json.strip("[")
    raw_json = raw_json.strip("]")
    
    with open(file_path, "a") as write_file:
        if month_name == "January":
            write_file.write("[")
        
        write_file.write(raw_json)
        
        if month_name != "DECEMBER":
            write_file.write(",")
        else:
            write_file.write("]")
    write_file.close()    

In [0]:
api_key = "5m6g-bqm4"

months = []
for month in calendar.month_name[1:]:
    months.append(month)
    months.append(month.upper())
    
file_path="C:/Documents/noise_rating.json"

for m in months:
    get_write_data(file_path, api_key, m)

In [0]:
f1=open("C:/Documents/noise_rating.json","r")
data1 = json.loads(f1.read())
f1.close()

In [0]:
#convert to a frame
frame = defaultdict(list)
empty_idx = []
for data in data1:
    #missing values for airline name
    if len(data.keys()) < 9:
        frame["airline"].append("EMPTY")
    for key in data:
        if key == "airline":
            if data[key] == "ERROR: #N/A":
                break
        frame[key].append(data[key])

In [0]:
length_dict = {key:len(value) for key, value in frame.items()}

In [0]:
#convert airlines with names in lower caps to all upper caps
new_airline = []
for i in frame["airline"]:
    val = i
    if i.isupper() == False:
        val = i.upper()
    new_airline.append(val)

v_index = [i for i, x in enumerate(new_airline) if x == "VOLGA\xadDNEPR"]

for i in v_index:
    new_airline[i] = "VOLGA-DNEPR"
    
frame["airline"] = new_airline

unique_airline = []
unique_code = []

for i, x in enumerate(frame["airline"]):
    if x not in unique_airline and x != "EMPTY":
        unique_airline.append(x)
        unique_code.append(frame["airline_code"][i])

In [0]:
#get the rows where the airline field is empty i.e. labelled "EMPTY"
empty_idx = [i for i, x in enumerate(new_airline) if x == "EMPTY"]
code_idx = []
retrieved_name =[]
for idx in empty_idx:
    code = frame["airline_code"][idx]
    if code in unique_code:
        match = unique_code.index(code)
        ins = idx, unique_airline[match]
        retrieved_name.append(ins)
        code_idx.append(idx)

#update the names retrieved, 
for idx, name in retrieved_name:
    frame["airline"][idx] = name

#delete the ones that were not as there is not enough examples
discard_idx = [i for i in empty_idx if i not in code_idx]
columns = frame.keys()
while("EMPTY" in frame["airline"]):
    next_empty = frame["airline"].index("EMPTY")
    for col in columns:
        del frame[col][next_empty]
    

In [0]:
rating_table = pd.DataFrame(frame)

In [0]:
rating_table[rating_table["airline"]== "EMPTY"]

,airline,airline_code,exceedances_per_1000_operations,month,noise_exceedance_quality_rating_score,noise_exceedance_rating,total_noise_exceedances,total_operations_per_month,year


In [0]:
months_format = calendar.month_name[1:]

In [0]:
#modify e.g. 'APRIL' to 'April'
for m in months_format:
    month_idx = rating_table.index[rating_table["month"] == m.upper()]
    
    if len(month_idx > 0):
        for idx in month_idx:
            rating_table.at[idx, "month"] = m

In [0]:
#remove "noise_exceeding_rating"
redun_feature = "noise_exceedance_rating"
cols = [i for i in rating_table.columns if i not in redun_feature]
rating_table = rating_table[cols]

In [0]:
#change data type 
dataset_idx = rating_table.index.tolist()

In [0]:
for idx in dataset_idx:
    exceed_1000_val = rating_table.iloc[idx]["exceedances_per_1000_operations"]
    rating_table.at[idx, "exceedances_per_1000_operations"] = int(exceed_1000_val)

In [0]:
for idx in dataset_idx:
    noise_score_val = rating_table.iloc[idx]["noise_exceedance_quality_rating_score"]
    rating_table.at[idx, "noise_exceedance_quality_rating_score"] = float(noise_score_val)

In [0]:
for idx in dataset_idx:
    tot_noise_val = rating_table.iloc[idx]["total_noise_exceedances"]
    rating_table.at[idx, "total_noise_exceedances"] = int(tot_noise_val)

In [0]:
for idx in dataset_idx:
    total_op_val = rating_table.iloc[idx]["total_operations_per_month"]
    rating_table.at[idx, "total_operations_per_month"] = int(total_op_val)

In [0]:
for idx in dataset_idx:
    year_val = rating_table.iloc[idx]["year"]
    rating_table.at[idx, "year"] = int(year_val)

In [0]:
#define function to get average ratio of tot_noise_excee/tot_operations_month for the year
def get_avg_ratio_year(airline_name, year, max_exceedance):
    global rating_table
    
    instances = rating_table[(rating_table["airline"] == airline_name) & (rating_table["year"] == year) & \
                         (rating_table["exceedances_per_1000_operations"] <= max_exceedance)]
    if len(instances) < 1:
        return 0
    
    total_ratio = 0
    
    for i in instances.index.tolist():
        total_ratio += (rating_table.iloc[i]["total_noise_exceedances"] / rating_table.iloc[i]["total_operations_per_month"])
    
    return total_ratio / len(instances)

In [0]:
#define function to get average ratio from all the years, if not available discard airline
def get_avg_ratio_allyears(airline_name, max_exceedance): #make rating_table global
    
    global rating_table
    instances = rating_table[(rating_table["airline"] == airline_name)  & (rating_table["exceedances_per_1000_operations"] <= max_exceedance)]
    
    if len(instances) < 1:
        return 0
    
    #store the airlines that have no instances where no_exceed > no_operations. We dont have enough data hence will be discarded
    total_ratio = 0

    for i in instances.index.tolist():
        total_ratio += (rating_table.iloc[i]["total_noise_exceedances"] / rating_table.iloc[i]["total_operations_per_month"])
        
    return total_ratio / len(instances)


In [0]:
def update_tot_noise(instances_above, instances_idx, ratio): #update total_noise_exceedance
    
    global rating_table
    new_calc = []
    
    #calculate new total_noise exceedance for each instance
    for j in instances_above["total_operations_per_month"]:
        new_calc.append(int(j * ratio))
    
    #change to new value in main data frame
    for i in range(len(instances_idx)):
        rating_table.at[instances_idx[i], "total_noise_exceedances"] = new_calc[i]

In [0]:
def update_ex_1000(airline_name): #update exceedance_per_1000_operations
    global rating_table
    
    instances_idx = rating_table[(rating_table["airline"] == airline_name) & \
                                 (rating_table["exceedances_per_1000_operations"] > max_exceedance)].index.tolist()
    
    for idx in instances_idx:
        new_val = (rating_table.iloc[idx]["total_noise_exceedances"] / rating_table.iloc[idx]["total_operations_per_month"]) * 1000
        rating_table.at[idx, "exceedances_per_1000_operations"] = new_val #int(new_val)
    

In [0]:
def update_noise_score(airline_name):
    
    

In [0]:
#max
rating_table["exceedances_per_1000_operations"].max()
#ma["exceedances_per_1000_operations"].max()

'992'

In [0]:
#unique = above["airline"].unique()
elel = rating_table[(rating_table["total_noise_exceedances"] < rating_table["total_operations_per_month"])]
#me =above[above["airline"] == "SINGAPORE AIRLINES"]
#(rating_table["airline"] == "SINGAPORE AIRLINES")
#elel[elel["year"] == 2008]
#me["year"].unique()
rating_table[rating_table["year"] == 2009]

,airline,airline_code,exceedances_per_1000_operations,month,noise_exceedance_quality_rating_score,total_noise_exceedances,total_operations_per_month,year
0,NETJETS AVIATION,EJA,2,January,9.99,1,637,2006
1,SKYWEST,SKW,2,January,9.99,15,6752,2006
2,US AIRWAYS,USA,5,January,9.97,2,388,2006
3,HORIZON AIR,QXE,6,January,9.97,1,168,2006
4,LUFTHANSA,DLH,9,January,9.95,1,108,2006
5,FRONTIER AIRLINES,FFT,10,January,9.95,3,289,2006
6,US AIRWAYS,AWE,14,January,9.93,10,731,2006
7,AIRTRAN AIRWAYS,TRS,15,January,9.92,1,67,2006
8,AIR FRANCE,AFR,16,January,9.92,1,62,2006
9,HAWAIIAN AIRLINES,HAL,16,January,9.92,1,62,2006


In [0]:
max_exceedance = 1000
#find instances where no_exceed > no_operations
above = rating_table[rating_table["exceedances_per_1000_operations"] > max_exceedance]
discard_airline = []
pepe = ["SINGAPORE AIRLINES"]
for name in above["airline"].unique():
    #get the instances from dtaframe returned with exceedance_per_op > 1000
    airline_instances = above[above["airline"] == name]
    avg_ratio = 0
    #get the years in which the error occurs
    for year in airline_instances["year"].unique():
        avg_year_ratio = get_avg_ratio_year(name, year, max_exceedance)
        
        if avg_year_ratio == 0:
            avg_ratio = get_avg_ratio_allyears(name, max_exceedance)

            if avg_ratio > 0:
                break
            elif avg_ratio == 0: #if there is not enough data for the airline, discard
                discard_airline.append(name)
                break
        else:
            print(avg_year_ratio)
            year_instances = airline_instances[airline_instances["year"] == year]
            update_tot_noise(year_instances, year_instances.index.tolist(), avg_year_ratio)
    
    #If there is not enough examples to calculate avg ratio for each year, use avg_ratio from all "valid" instances
    if avg_ratio > 0:
        update_tot_noise(airline_instances, airline_instances.index.tolist(), avg_ratio)  
    
    #update per 1000 operations if total_noise_exceedance is updated
    update_check = rating_table[(rating_table["airline"]== name) & \
                                (rating_table["total_noise_exceedances"] > rating_table["total_operations_per_month"])]
    
    if len(update_check) == 0:
        update_ex_1000(name)
        #update_noise_score(name)

In [0]:
#answ: Royal Air, Polet Airlines Cargo, Qantas Airway
frame = {}
first = data1[0]
for key in first:
    frame[key] = first[key]


for i in range(1, len(data1)):
    row= data1[i]
    for key in row:
        col_data = []
        val = frame[key]
        if type(val) != str and len(val) > 1:
            for j in val:
                col_data.append(j)
        else:
            col_data.append(val)
        col_data.append(row[key])
        frame[key] = col_data

1998

In [0]:
46:96(40), 40:82(238), 55:114(532)
frame ={}
for data in data2:
    for key in data:
        #get the
        frame.setdefault(key, []).append(data[key])
complaint_table = pd.DataFrame(frame)
complaint_table

max_exceedance = 1000

#change data type in exceedances_per_1000_operations to int
rating_table["exceedances_per_1000_operations"] = [int(v) for v in rating_table["exceedances_per_1000_operations"]]

#get the index where example exceeds 1000
above_idx = rating_table.index[rating_table["exceedances_per_1000_operations"] > max_exceedance]

exceed_op = rating_table["exceedances_per_1000_operations"]

#change values over to 1000
for i in above_idx:
    exceed_op.iloc[i] = max_exceedance

rating_table["exceedances_per_1000_operations"] = exceed_op

#find instances where no_exceed > no_operations
above = rating_table[rating_table["exceedances_per_1000_operations"] > max_exceedance]

eva = rating_table[(rating_table["airline"] == "EVA AIRWAYS")  & (rating_table["exceedances_per_1000_operations"] < max_exceedance)]
total_ratio = 0

for i in eva.index.tolist():
    #print(i)
    total_ratio += (rating_table.iloc[i]["total_noise_exceedances"] / rating_table.iloc[i]["total_operations_per_month"])
    
avg_ratio = total_ratio / len(eva)

airline = above[above["airline"] == "EVA AIRWAYS"]
new_calc = []
for j in airline["total_operations_per_month"]:
    new_calc.append(int(j * avg_ratio))

for i in range(len(airline)):
    locs = airline.index.tolist()
    loc = locs[i]
    rating_table.at[loc, "total_noise_exceedances"] = new_calc[i]
    

NameError: name 'data2' is not defined

In [0]:
max_exceedance = 1000
#find instances where no_exceed > no_operations
above = rating_table[rating_table["exceedances_per_1000_operations"] > max_exceedance]
#above["airline"].unique() -> SINGAPORE AIRLINES
singapore = rating_table[(rating_table["airline"] == "SINGAPORE AIRLINES") & (rating_table["year"] == 2006) & \
                         (rating_table["exceedances_per_1000_operations"] < max_exceedance)]

#store the exceedance value for airline above threhold for the year
airline_above = above[above["airline"] == "KALITTA AIR"]
#airline_above["year"].unique()
airline_above


,airline,airline_code,exceedances_per_1000_operations,month,noise_exceedance_quality_rating_score,total_noise_exceedances,total_operations_per_month,year
35,KALITTA AIR,CKS,1091,January,4.43,24,22,2006
359,KALITTA AIR,CKS,1333,January,3.24,8,6,2015
395,KALITTA AIR,CKS,2167,January,1.33,39,18,2016
439,KALITTA AIR,CKS,1857,January,1.98,13,7,2017
872,KALITTA AIR,CKS,1250,February,3.87,5,4,2016
1312,KALITTA AIR,CKS,1083,March,5.15,26,24,2015
1353,KALITTA AIR,CKS,1107,March,0,31,28,2016
1392,KALITTA AIR,CKS,2278,March,0.24,41,18,2017
1520,KALITTA AIR,CKS,1871,April,0,58,31,2007
1810,KALITTA AIR,CKS,3650,April,0,73,20,2015
